In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import time
from collections import OrderedDict

Laden des Trainingsset und Konvertierung in Pytorch Tensoren

In [2]:
df_trainingset_features = pd.read_csv(r"D:\Uni\BA\Uploadable\Datasets\df_trainingsset_features")
df_trainingset_labels = pd.read_csv(r"D:\Uni\BA\Uploadable\Datasets\df_trainingsset_labels")
tensor_trainingset_features = torch.tensor(df_trainingset_features.values, dtype = torch.float)
tensor_trainingset_labels = torch.tensor(df_trainingset_labels.values, dtype = torch.float).view(-1,1)
print(tensor_trainingset_features)
print(tensor_trainingset_labels)

tensor([[ 0.8219,  0.8524, -0.5941,  0.6035],
        [-1.6570, -1.6333,  2.3839, -0.3424],
        [ 1.7289,  0.3286, -0.7481,  1.0844],
        ...,
        [-2.6214,  1.2636, -0.3096, -1.6086],
        [-1.3664,  0.1487, -0.3106,  0.0772],
        [-1.3305, -1.5281,  2.2923,  0.2935]])
tensor([[0.],
        [1.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]])


1) Neuronales Netzwerk definieren. Hier das 4_32_32_1 Netzwerk

In [3]:
class BankNodes(nn.Module):
    def __init__(self, config, is_training=True):
        super(BankNodes, self).__init__()
        self.config = config
        self.training = is_training

        # Define the layers using OrderedDict
        layers = OrderedDict()
        layers['fc1'] = nn.Linear(in_features=config["input_size"], out_features=config["hiddenlayer_size1"])
        layers['relu1'] = nn.ReLU()
        layers['fc2'] = nn.Linear(in_features=config["hiddenlayer_size1"], out_features=config["hiddenlayer_size2"])
        layers['relu2'] = nn.ReLU()
        layers['fc3'] = nn.Linear(in_features=config["hiddenlayer_size2"], out_features=config["outputlayer_size"])
        layers['sig3'] = nn.Sigmoid()

        # Create the neural network using the layers
        self.network = nn.Sequential(layers)

    def forward(self, x):
        # Forward pass through the network
        return self.network(x)

Model initialisieren

In [4]:
config = {"input_size": 4, "hiddenlayer_size1": 32, "hiddenlayer_size2": 32, "outputlayer_size": 1} # Adjust these values as needed
model = BankNodes(config)

Verlust Funktion festlegen

In [5]:
lossFunc = nn.BCELoss()
optFunc = torch.optim.Adam(model.parameters(), lr=0.001)

Epochen sowie Batchgröße Festlegen
30x32 = 960 Daten werden beim Lernen durchgefüttert

In [6]:
epochs = 10
batch_size = 32

Trainingsloop

In [7]:
for epoch in range(epochs):

    for i in range(0, tensor_trainingset_features.size(0), batch_size):
        inputs = tensor_trainingset_features[i: i + batch_size]
        labels = tensor_trainingset_labels[i: i + batch_size]
        
        optFunc.zero_grad()
        outputs = model(inputs)
        loss = lossFunc(outputs, labels)
        loss.backward()
        optFunc.step()
       
        

    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")    

Epoch [1/10], Loss: 0.5984
Epoch [2/10], Loss: 0.3997
Epoch [3/10], Loss: 0.2432
Epoch [4/10], Loss: 0.1571
Epoch [5/10], Loss: 0.1025
Epoch [6/10], Loss: 0.0672
Epoch [7/10], Loss: 0.0472
Epoch [8/10], Loss: 0.0348
Epoch [9/10], Loss: 0.0266
Epoch [10/10], Loss: 0.0207


Evaluierung des Trainierten Netzes
Laden des Evaluierungsdatensatzes

In [8]:
df_validationset_features = pd.read_csv(r"D:\Uni\BA\Uploadable\Datasets\df_validationset_features")
df_validationset_labels = pd.read_csv(r"D:\Uni\BA\Uploadable\Datasets\df_validationset_labels")
tensor_validationset_features = torch.tensor(df_validationset_features.values, dtype = torch.float)
tensor_validationset_labels = torch.tensor(df_validationset_labels.values, dtype = torch.float).view(-1,1)
#print(tensor_validationset_features)
#print(tensor_validationset_labels)

Evaluierung starten

In [28]:
#print(tensor_validationset_features)
#print(tensor_validationset_labels)

with torch.no_grad():
    model.eval()
     #Start timer for eval
    time_start = time.time()
    
    validation_outputs = np.round(model(tensor_validationset_features))
    
    #Stop time for performance mesaurement
    time_end = time.time()
    timetotal = time_end - time_start
    total_frames = 1
    fps = float(total_frames / timetotal)
    print(
        "FPS=%.2f, total frames = %.2f , time=%.6f seconds"
        % (fps, total_frames, timetotal)
    )
    
    network_accuracy = (validation_outputs == tensor_validationset_labels).float().mean()
    print("O: ", validation_outputs)
    print("E: ", tensor_validationset_labels )
    print(torch.eq(validation_outputs, tensor_validationset_labels))
    accurate_predictions = torch.eq(validation_outputs, tensor_validationset_labels).to(torch.float32)
    network_accuracy = accurate_predictions.mean().item()
    print("Accuracy:", network_accuracy)


FPS=1000.55, total frames = 1.00 , time=0.000999 seconds
O:  tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
 

In [26]:
#model_path = r"D:\Uni\BA\ownModel\4_32_32_1\weights\pretrained_banknode432321.pth"
#torch.save(model.state_dict(), model_path)